In [ ]:
import numpy as np
from numpy import mean, absolute
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from scipy.linalg import pinv

from sklearn.metrics import roc_curve

import pandas as pd

In [ ]:
###########################################################################
# 1. Anomaly detectors
###########################################################################

# The euclideanTrain and euclideanScore functions comprise the
# Euclidean anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector.  This mean vector is returned as the
# detection model.  During scoring, the detector takes the detection
# model and a new set of password-timing vectors (also encoded as rows
# in a matrix) and calculates the squared Euclidean distance between
# the mean vector and each of the new password-timing vectors.  These
# scores are returned in a vector whose length is equal to the number
# of password-timing vectors in the scoring matrix.

def euclideanTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0)}
    return dmod


def euclideanScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))

    scores = np.sum(np.square(YScore - meanMatrix), axis=1)

    return scores


# The manhattanTrain and manhattanScore functions comprise the
# Manhattan anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector.  This mean vector is returned as the
# detection model.  During scoring, the detector takes the detection
# model and a new set of password-timing vectors (also encoded as rows
# in a matrix) and calculates the Manhattan distance between the mean
# vector and each of the new password-timing vectors.  These scores
# are returned in a vector whose length is equal to the number of
# password-timing vectors in the scoring matrix.

def manhattanTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0)}
    return dmod


def manhattanScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))

    scores = np.sum(np.abs(YScore - meanMatrix), axis=1)

    return scores


# Scaled Manhattan distance
def mad(data, axis=None):
    return mean(absolute(data - mean(data, axis)), axis)

def manhattanScaledTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0), 'mad': mad(YTrain,0)}
    return dmod


def manhattanScaledScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]
    
    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))
    scores = np.sum(np.abs(YScore - meanMatrix)/dmod["mad"].T, axis=1)
    return scores


# The mahalanobisTrain and mahalanobisScore functions comprise the
# Mahalanobis anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector and also the inverse of the covariance
# matrix.  This vector and matrix are returned as the detection model.
# During scoring, the detector takes the detection model and a new set
# of password-timing vectors (also encoded as rows in a matrix) and
# calculates the squared Mahalanobis distance between the mean vector
# and each of the new password-timing vectors.  These scores are
# returned in a vector whose length is equal to the number of
# password-timing vectors in the scoring matrix.

def mahalanobisTrain(YTrain):
    dmod = {
        'mean': np.mean(YTrain, axis=0), 
        'covInv': pinv(np.cov(YTrain, rowvar=False))}
    return dmod


def mahalanobisR(X, meanCol, IC):
    m = []
    for i in range(X.shape[0]):
        m.append(distance.mahalanobis(X[i], meanCol, IC) ** 2)
    return(m)


def mahalanobisScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    scores = mahalanobisR(YScore, dmod['mean'], dmod['covInv'])

    return scores


detectorSet = {
    'Euclidean': {
        'train': euclideanTrain,
        'score': euclideanScore
    },
    'Manhattan': {
        'train': manhattanTrain,
        'score': manhattanScore
    },
    'ManhattanScaled':{
        'train': manhattanScaledTrain,
        'score': manhattanScaledScore
    },
    'Mahalanobis': {
        'train': mahalanobisTrain,
        'score': mahalanobisScore
    }
}


In [ ]:
###########################################################################
# 2. Evaluation procedures
###########################################################################

# The calculateEqualError function takes a set of user scores and
# impostor scores, makes an ROC curve using the ROCR functionality,
# and then geometrically calculates the point at which the miss and
# false-alarm (i.e., false-negative and false-positive) rates are
# equal.

def calculateEqualError(userScores, impostorScores):
    
    predictions = np.concatenate((userScores, impostorScores), axis=None)
    labels = [0] * len(userScores) + [1] * len(impostorScores)

    fpr, tpr, thresholds = roc_curve(labels, predictions)

    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]


    # print(eer)
    # print(thresholds)
    #dists = tpr - fpr
    # print(dists)
    #idx1 = next(i for i, dist in enumerate(dists) if dist == min(dists[dists >= 0]))
    #idx2 = next(i for i, dist in enumerate(dists) if dist == max(dists[dists < 0]))
    # print(idx1)
    #assert len([idx1]) == 1
    #assert len([idx2]) == 1
    #assert abs(idx1 - idx2) == 1

    #x = [tpr[idx1], fpr[idx1]]
    #y = [tpr[idx2], fpr[idx2]]
    #a = (x[0] - x[1]) / (y[1] - x[1] - y[0] + x[0])
    #eer = x[0] + a * (y[0] - x[0])

    return eer


def evaluateSubject(train, eval, evalSubject, detectorTrain, detectorScore):
    # Extract the training, user scoring, and impostor scoring matrices
    # for the subject. The training matrix is the first 200 password
    # repetitions for the subject, corresponding to the first 4 sessions
    # of passwords. The user scoring matrix is the last 200 password
    # repetitions, and the impostor scoring matrix is the first 5
    # repetitions from all the other subjects.

    YTrain = train.loc[(train['subject'] == evalSubject) & (
        train['sessionIndex'] <= 4), ~train.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    YScore0 = eval.loc[(eval['subject'] == evalSubject) & (
        eval['sessionIndex'] > 4), ~eval.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    YScore1 = train.loc[(train['subject'] != evalSubject) & (train['sessionIndex'] == 1) & (
        train['rep'] <= 5), ~train.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    # Run the training and scoring
    dmod = detectorTrain(YTrain)
    userScores = detectorScore(dmod, YScore0)
    impostorScores = detectorScore(dmod, YScore1)

    eer = calculateEqualError(userScores, impostorScores)

    return eer


In [ ]:
###########################################################################
# 3. Main procedure
###########################################################################
def eval(train, eval):
    # Retrieve the list of subjects
    subjects = sorted(train["subject"].unique())

    training_set = train
    eval_set = eval


    # For each of the detectors, evaluate the detector on each subject,
    # and record the equal-error rates in a dictionary.

    eers = {}

    for detectorName, detector in detectorSet.items():

        #print(f"Evaluating the {detectorName} detector")
        detectorTrain = detector['train']
        detectorScore = detector['score']

        eers[detectorName] = np.full(len(subjects), np.nan)

        for i, subject in enumerate(subjects):
            eer = evaluateSubject(training_set, eval_set, subject,
                                detectorTrain=detectorTrain,
                                detectorScore=detectorScore)

            eers[detectorName][i] = eer
            #print(f"  {i+1}/{len(subjects)}: {eer}", end='\r')

        #print(f"  average equal-error: {np.mean(eers[detectorName])}")

    #print("Tabulating results:")

    eers = pd.DataFrame(eers, index=subjects)

    res = pd.DataFrame({'eer.mean': eers.mean(), 'eer.sd': eers.std()})
    res = res.round(decimals=3)

    print(res, "\n")


In [ ]:
# evaluation routine
original = pd.read_csv('./DSL-StrongPasswordData.csv')

def eval_set(browser_data_set: pd.DataFrame):

    print("Original - Browser")
    eval(train=original, eval=browser_data_set)

    print("Browser - Original")
    eval(train=browser_data_set, eval=original)

    print("Browser - Browser")
    eval(train=browser_data_set, eval=browser_data_set)

print("Original - Original")
eval(train=original, eval=original)

In [ ]:
# Firefox isolated
firefox_isolated = pd.read_csv('./firefox/password_data_firefox_isolated/full_data_set.csv')
#eval_set(firefox_isolated)

In [ ]:
# Firefox unisolated
firefox_unisolated = pd.read_csv('./firefox/password_data_firefox_unisolated/full_data_set.csv')
#eval_set(firefox_unisolated)

In [ ]:
# Firefox isolated warmup
firefox_isolated_warmup = pd.read_csv('./firefox/password_data_firefox_isolated_warmup/full_data_set.csv')
eval_set(firefox_isolated_warmup)

In [ ]:
# Firefox unisolated warmup
firefox_unisolated_warmup = pd.read_csv('./firefox/password_data_firefox_unisolated_warmup/full_data_set.csv')
eval_set(firefox_unisolated_warmup)

In [ ]:
# Edge isolated
edge_isolated = pd.read_csv('./edge/password_data_edge_isolated/full_data_set.csv')
#eval_set(edge_isolated)

In [ ]:
# Edge unisolated
edge_unisolated = pd.read_csv('./edge/password_data_edge_unisolated/full_data_set.csv')
#eval_set(edge_unisolated)

In [ ]:
# Edge isolated warmup
edge_isolated_warmup = pd.read_csv('./edge/password_data_edge_isolated_warmup/full_data_set.csv')
eval_set(edge_isolated_warmup)

In [ ]:
# Edge unisolated warmup
edge_unisolated_warmup = pd.read_csv('./edge/password_data_edge_unisolated_warmup/full_data_set.csv')
eval_set(edge_unisolated_warmup)

In [ ]:
# Chromium isolated
chromium_isolated = pd.read_csv('./chromium/password_data_chromium_isolated/full_data_set.csv')
#eval_set(chromium_isolated)

In [ ]:
# Chromium unisolated
chromium_unisolated = pd.read_csv('./chromium/password_data_chromium_unisolated/full_data_set.csv')
#eval_set(chromium_unisolated)

In [ ]:
# Chromium isolated warmup
chromium_isolated_warmup = pd.read_csv('./chromium/password_data_chromium_isolated_warmup/full_data_set.csv')
eval_set(chromium_isolated_warmup)

In [ ]:
# Chromium unisolated warmup
chromium_unisolated_warmup = pd.read_csv('./chromium/password_data_chromium_unisolated_warmup/full_data_set.csv')
eval_set(chromium_unisolated_warmup)

In [ ]:
# Chrome isolated
chrome_isolated = pd.read_csv('./chrome/password_data_chrome_isolated/full_data_set.csv')
#eval_set(chrome_isolated)

In [ ]:
# Chrome unisolated
chrome_unisolated = pd.read_csv('./chrome/password_data_chrome_unisolated/full_data_set.csv')
#eval_set(chrome_unisolated)

In [ ]:
# Chrome isolated warmup
chrome_isolated_warmup = pd.read_csv('./chrome/password_data_chrome_isolated_warmup/full_data_set.csv')
eval_set(chrome_isolated_warmup)

In [ ]:
# Chrome unisolated warmup
chrome_unisolated_warmup = pd.read_csv('./chrome/password_data_chrome_unisolated_warmup/full_data_set.csv')
eval_set(chrome_unisolated_warmup)

In [ ]:
# Tor unisolated
tor_unisolated = pd.read_csv('./tor/password_data_tor_unisolated/full_data_set.csv')
#eval_set(tor_unisolated)

In [ ]:
# Tor unisolated warmup
tor_unisolated_warmup = pd.read_csv('./tor/password_data_tor_unisolated_warmup/full_data_set.csv')
eval_set(tor_unisolated_warmup)

In [ ]:
# Safari isolated warm up
safari_isolated_warm_up = pd.read_csv(f'./safari/password_data_safari_isolated_warmup/full_data_set.csv')
eval_set(safari_isolated_warm_up)

In [ ]:
# Safari unisolated warm up
safari_unisolated_warm_up = pd.read_csv(f'./safari/password_data_safari_unisolated_warmup/full_data_set.csv')
eval_set(safari_unisolated_warm_up)

In [ ]:
# Safari isolated
safari_isolated_warm_up = pd.read_csv('./safari/password_data_safari_isolated/full_data_set.csv')
#eval_set(safari_isolated_warm_up)

In [ ]:
# Safari unisolated
safari_isolated_warm_up = pd.read_csv('./safari/password_data_safari_unisolated/full_data_set.csv')
#eval_set(safari_isolated_warm_up)

In [ ]:
# Firefox isolated warm up | resistFingerprinting
firefox_isolated_resist = pd.read_csv('./firefox_resistFP/firefox_isolated_resistFP/full_data_set.csv')
eval_set(firefox_isolated_resist)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting
firefox_unisolated_resist = pd.read_csv('./firefox_resistFP/firefox_unisolated_resistFP/full_data_set.csv')
eval_set(firefox_unisolated_resist)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 20ms
firefox_unisolated_resist_20ms = pd.read_csv('./firefox_resistFP_20/firefox_unisolated_resistFP_20ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_20ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 33ms
firefox_unisolated_resist_33ms = pd.read_csv('./firefox_resistFP_33/firefox_unisolated_resistFP_33ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_33ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 40ms
firefox_unisolated_resist_40ms = pd.read_csv('./firefox_resistFP_40/firefox_unisolated_resistFP_40ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_40ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 60ms
firefox_unisolated_resist_60ms = pd.read_csv('./firefox_resistFP_60/firefox_unisolated_resistFP_60ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_60ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 80ms
firefox_unisolated_resist_80ms = pd.read_csv('./firefox_resistFP_80/firefox_unisolated_resistFP_80ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_80ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 100ms
firefox_unisolated_resist_100ms = pd.read_csv('./firefox_resistFP_100/firefox_unisolated_resistFP_100ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_100ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 120ms
firefox_unisolated_resist_120ms = pd.read_csv('./firefox_resistFP_120/firefox_unisolated_resistFP_120ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_120ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 140ms
firefox_unisolated_resist_140ms = pd.read_csv('./firefox_resistFP_140/firefox_unisolated_resistFP_140ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_140ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 160ms
firefox_unisolated_resist_160ms = pd.read_csv('./firefox_resistFP_160/firefox_unisolated_resistFP_160ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_160ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 180ms
firefox_unisolated_resist_180ms = pd.read_csv('./firefox_resistFP_180/firefox_unisolated_resistFP_180ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_180ms)

In [ ]:
# Firefox unisolated warm up | resistFingerprinting 200ms
firefox_unisolated_resist_200ms = pd.read_csv('./firefox_resistFP_200/firefox_unisolated_resistFP_200ms/full_data_set.csv')
eval_set(firefox_unisolated_resist_200ms)

In [ ]:
# Firefox unisolated resistFingerprinting NJ
firefox_unisolated_resist_NJ = pd.read_csv('./firefox_resistFP_NJ/firefox_unisolated_resistFP_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 20ms NJ
firefox_unisolated_resist_20ms_NJ = pd.read_csv('./firefox_resistFP_20_NJ/firefox_unisolated_resistFP_20ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_20ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 33ms NJ
firefox_unisolated_resist_33ms_NJ = pd.read_csv('./firefox_resistFP_33_NJ/firefox_unisolated_resistFP_33ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_33ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 40ms NJ
firefox_unisolated_resist_40ms_NJ = pd.read_csv('./firefox_resistFP_40_NJ/firefox_unisolated_resistFP_40ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_40ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 60ms NJ
firefox_unisolated_resist_60ms_NJ = pd.read_csv('./firefox_resistFP_60_NJ/firefox_unisolated_resistFP_60ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_60ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 80ms NJ
firefox_unisolated_resist_80ms_NJ = pd.read_csv('./firefox_resistFP_80_NJ/firefox_unisolated_resistFP_80ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_80ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 100ms NJ
firefox_unisolated_resist_100ms_NJ = pd.read_csv('./firefox_resistFP_100_NJ/firefox_unisolated_resistFP_100ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_100ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 120ms NJ
firefox_unisolated_resist_120ms_NJ = pd.read_csv('./firefox_resistFP_120_NJ/firefox_unisolated_resistFP_120ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_120ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 140ms NJ
firefox_unisolated_resist_140ms_NJ = pd.read_csv('./firefox_resistFP_140_NJ/firefox_unisolated_resistFP_140ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_140ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 160ms NJ
firefox_unisolated_resist_160ms_NJ = pd.read_csv('./firefox_resistFP_160_NJ/firefox_unisolated_resistFP_160ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_160ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 180ms NJ
firefox_unisolated_resist_180ms_NJ = pd.read_csv('./firefox_resistFP_180_NJ/firefox_unisolated_resistFP_180ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_180ms_NJ)

In [ ]:
# Firefox unisolated resistFingerprinting 200ms NJ
firefox_unisolated_resist_200ms_NJ = pd.read_csv('./firefox_resistFP_200_NJ/firefox_unisolated_resistFP_200ms_NJ/full_data_set.csv')
eval_set(firefox_unisolated_resist_200ms_NJ)